In [54]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (112 kB/s)

In [55]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-01-17 02:06:54--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  6.01MB/s    in 0.2s    

2021-01-17 02:06:54 (6.01 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [56]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [57]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   35471030|R1C5OK9AND7PRP|B00AHK07V0|     175130663|Hunter's Moon (A ...|Digital_Ebook_Pur...|          5|            0|          0|   N|                Y|Shugak is like a ...|This is the most ...| 2015-08-31|
|         US|   26579324|R333RNBQMPVUFT|B014085WTQ|     859232728|Flying Toward For...|Digital_Ebook_Pur

### Create DataFrames to match tables

In [58]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame

df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   35471030|R1C5OK9AND7PRP|B00AHK07V0|     175130663|Hunter's Moon (A ...|Digital_Ebook_Pur...|          5|            0|          0|   N|                Y|Shugak is like a ...|This is the most ...| 2015-08-31|
|         US|   26579324|R333RNBQMPVUFT|B014085WTQ|     859232728|Flying Toward For...|Digital_Ebook_Pur

In [59]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   41341751|            18|
|   14582882|             1|
|   15158447|             7|
|    3070853|            16|
|   26248773|           511|
|   50058978|             1|
|   16667845|             3|
|   14217455|             1|
|   14754839|             6|
|   35311351|             7|
|   40182046|             1|
|   47574496|             1|
|   43133249|            10|
|   44269342|           307|
|   20688276|             1|
|   12819130|             4|
|    1154193|            21|
|   52427235|             6|
|   15948906|             1|
|   22996409|            13|
+-----------+--------------+
only showing top 20 rows



In [60]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id","product_title"]).drop_duplicates(["product_id"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0439394422|school house rock...|
|9861064222|Black Nintendo Wi...|
|B00000K1XN|        WWF Attitude|
|B00001OX40|   Neo Cherry Master|
|B00001QEQD|     USCF Chess - PC|
|B00001XDTO|        Yoda Stories|
|B00001ZWQ0|Legend of the Riv...|
|B0000296ZL|      Game Boy Color|
|B00002SFMS|     Boss Rally - PC|
|B00002SSYX|Ristar (Sega Game...|
|B00002STA7|    NFL Football '94|
|B00002STH3|           Extreme G|
|B00002STJC|                 MDK|
|B00002STWV|Virtual Open Tenn...|
|B00002SW1K|        World Heroes|
|B00002SW49|Lawnmower Man - S...|
|B000035XH1|RoboCop vs. Termi...|
|B000035XYM|Alien vs. Predato...|
|B000035Y3U|Top Gear 2 - Nint...|
|B000046S3Z|           Crystalis|
+----------+--------------------+
only showing top 20 rows



In [61]:

# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(['review_id','customer_id','product_id','product_parent', to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1C5OK9AND7PRP|   35471030|B00AHK07V0|     175130663| 2015-08-31|
|R333RNBQMPVUFT|   26579324|B014085WTQ|     859232728| 2015-08-31|
|R2A2K0GTNSKXM1|    9152364|B0113MFPN0|     411472516| 2015-08-31|
|R365LAQ9REV876|   52087605|B00T8RIK7G|     353828021| 2015-08-31|
|R294OF3SNH6SWZ|   46454660|B00EMXBDMA|     803172158| 2015-08-31|
|R2OMRH58WYEEP9|   33168429|B00GL3OJIY|     325772386| 2015-08-31|
| RBFXYUJZIH5JR|   18546798|B00RW30QE8|     236508296| 2015-08-31|
|R2K2RSBNV42HU4|   47463409|B0101AE85Q|     924272105| 2015-08-31|
|R2CLRFFJ5HJSU3|   11767467|B00IO7QAI2|     393059633| 2015-08-31|
| RS1O7V45AADDO|   14241457|B00PM995TG|     529331494| 2015-08-31|
| RYD2AF3KUC4OL|   52821351|B00K4C4GVI|     310020786| 2015-08-31|
| RHQTCPUF1CXY2|   14416837|B00PUSKG8I|     683060701| 2015-08

In [62]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1C5OK9AND7PRP|          5|            0|          0|   N|                Y|
|R333RNBQMPVUFT|          5|            0|          0|   N|                Y|
|R2A2K0GTNSKXM1|          5|            0|          0|   N|                N|
|R365LAQ9REV876|          3|            0|          0|   N|                Y|
|R294OF3SNH6SWZ|          4|            0|          0|   N|                Y|
|R2OMRH58WYEEP9|          5|            0|          0|   N|                N|
| RBFXYUJZIH5JR|          5|            0|          0|   N|                Y|
|R2K2RSBNV42HU4|          5|            0|          0|   N|                N|
|R2CLRFFJ5HJSU3|          4|            0|          0|   N|                N|
| RS1O7V45AADDO|          4|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [63]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.cgetwbysm9dx.us-east-2.rds.amazonaws.com:5432/DataViz_Challenge"
config = {"user":"postgres", 
          "password": "eY4+H47q^q#8", 
          "driver":"org.postgresql.Driver"}

In [65]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

KeyboardInterrupt: ignored

In [ ]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [67]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

KeyboardInterrupt: ignored

In [66]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

KeyboardInterrupt: ignored